In [1]:
# -*- coding: utf-8
# Repórter Brasil (http://ruralometro.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Cria base com dívidas de deputados federais na Previdência Social e quais delas são do Funrural, baseado em informações da Procuradoria-Geral da Fazenda Nacional via LAI:  http://www.acessoainformacao.gov.br/assuntos/busca-de-pedidos-e-respostas/busca-de-pedidos-e-respostas
#

In [2]:
import pandas as pd

In [3]:
# Acessa base com dívidas no INSS

In [4]:
dividas_dep = pd.read_csv("dividas_deputados_ajustado_csv.csv",sep=';',encoding = 'latin_1', decimal = ',')

In [5]:
dividas_dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 10 columns):
CPF_Deputado                  106 non-null object
CPF_limpo                     106 non-null int64
Nome_Deputado                 106 non-null object
Vinculo                       106 non-null object
CNPJ_Devedor                  106 non-null object
CNPJ_limpo                    106 non-null int64
Nome_Devedor                  106 non-null object
Valores_situacao_Irregular    65 non-null float64
Valores_situacao_Regular      52 non-null float64
Total_Devido                  106 non-null float64
dtypes: float64(3), int64(2), object(5)
memory usage: 8.4+ KB


In [6]:
dividas_dep['CNPJ_limpo'] = dividas_dep.CNPJ_limpo.astype('float64')

In [7]:
# Cria base só com deputados eleitos em 2014 - via TSE

In [8]:
candidatos = pd.read_csv("candidatos_nome_urna_csv.csv",sep=';',encoding = 'latin_1')

In [9]:
eleitos = candidatos[(candidatos['Situacao'] == 'ELEITO POR QP') | (candidatos['Situacao'] == 'ELEITO POR MÉDIA') | (candidatos['Situacao'] == 'ELEITO')]

In [10]:
eleitos_d = eleitos[(eleitos['Cargo'] == 'DEPUTADO FEDERAL')]

In [11]:
eleitos_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 23 to 26219
Data columns (total 7 columns):
Estado            513 non-null object
Cargo             513 non-null object
Nome_candidato    513 non-null object
CPF               513 non-null int64
Nome_urna         513 non-null object
Partido           513 non-null object
Situacao          513 non-null object
dtypes: int64(1), object(6)
memory usage: 32.1+ KB


In [12]:
# Junta as duas bases e deixa só dívidas de deputados eleitos em 2014 - pelo CPFs

In [13]:
dividas = pd.merge(dividas_dep, eleitos_d, left_on='CPF_limpo', right_on='CPF')

In [14]:
dividas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 17 columns):
CPF_Deputado                  96 non-null object
CPF_limpo                     96 non-null int64
Nome_Deputado                 96 non-null object
Vinculo                       96 non-null object
CNPJ_Devedor                  96 non-null object
CNPJ_limpo                    96 non-null float64
Nome_Devedor                  96 non-null object
Valores_situacao_Irregular    59 non-null float64
Valores_situacao_Regular      47 non-null float64
Total_Devido                  96 non-null float64
Estado                        96 non-null object
Cargo                         96 non-null object
Nome_candidato                96 non-null object
CPF                           96 non-null int64
Nome_urna                     96 non-null object
Partido                       96 non-null object
Situacao                      96 non-null object
dtypes: float64(4), int64(2), object(11)
memory usage: 13.5+ K

In [15]:
# Acessa base com informações de dívidas do Funrural

In [16]:
# Funrural é a Previdência dos trabalhadores rurais 

In [17]:
funrural = pd.read_excel('DEVEDORES FUNRURAL ATUALIZADO PGFN.xlsx')

In [18]:
funrural.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8130 entries, 0 to 8129
Data columns (total 14 columns):
PSFN_PGFN               8129 non-null object
Regiao                  8129 non-null object
CNPJ_CEI_Tipo           8129 non-null object
CNPJ_Raiz               8129 non-null object
Razao_Social            8130 non-null object
Valor_principal         8130 non-null float64
Valor_TR_IPC_Poup       8130 non-null float64
Valor_Juros             8130 non-null float64
Valor_SELIC             8130 non-null float64
Valor_Encargo           8130 non-null float64
Valor_Multa_Oficio      8130 non-null float64
Valor_Selic_M_Oficio    8130 non-null float64
Vl_Multa_Mora           8130 non-null float64
Vl_Tot_Credito          8130 non-null float64
dtypes: float64(9), object(5)
memory usage: 889.3+ KB


In [19]:
# O problema aqui é que a lista de dívidas do Funrural tem o CNPJ raiz, ou seja, os oito primeiros números

In [20]:
# E a lista de dívidas em geral no INSS tem CNPJ com 14 números, então não dá para fazer um merge normal

In [21]:
# Uma das maneiras de resolver isso é isolar o CNPJ

In [22]:
pre_iguais = []

In [23]:
def cria(a, b, c, d, e, f, g, h, i, j, l, m, valor):
    pre_iguais.append([a, b, c, d, e, f, g, h, i, j, l, m, valor])
    return 

for num, row in dividas.iterrows():
    # é o CNPJ inteiro
    a = (row['CNPJ_Devedor'])
    # isola a raiz do CNPJ
    pega = (a[:10])
    c = (row['CPF_Deputado']) 
    d = (row['CPF_limpo'])
    e = (row['Nome_Deputado']) 
    f = (row['Vinculo']) 
    g = (row['CNPJ_limpo'])
    h = (row['Nome_Devedor'])
    i = (row['Valores_situacao_Irregular'])
    j = (row['Valores_situacao_Regular'])
    l = (row['Total_Devido'])
    for num2, row in funrural.iterrows():
        b = (row['CNPJ_Raiz'])
        m = (row['Vl_Tot_Credito'])
        if pega == b:
            valor = "Sim"
        else:
            valor = "Não"
        cria (a, b, c, d, e, f, g, h, i, j, l, m, valor)    

In [24]:
# Transforma em dataframe

In [25]:
iguais = pd.DataFrame(pre_iguais, columns = ['CNPJ_Devedor', 'CNPJ_Raiz', 'CPF_Deputado', 'CPF_limpo', 'Nome_Deputado', 'Vinculo', 'CNPJ_limpo', 'Nome_Devedor', 'Valores_situacao_Irregular', 'Valores_situacao_Regular', 'Total_Devido', 'Total_lista_Funrural', 'Mesma_empresa'])

In [26]:
iguais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780480 entries, 0 to 780479
Data columns (total 13 columns):
CNPJ_Devedor                  780480 non-null object
CNPJ_Raiz                     780384 non-null object
CPF_Deputado                  780480 non-null object
CPF_limpo                     780480 non-null int64
Nome_Deputado                 780480 non-null object
Vinculo                       780480 non-null object
CNPJ_limpo                    780480 non-null float64
Nome_Devedor                  780480 non-null object
Valores_situacao_Irregular    479670 non-null float64
Valores_situacao_Regular      382110 non-null float64
Total_Devido                  780480 non-null float64
Total_lista_Funrural          780480 non-null float64
Mesma_empresa                 780480 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 77.4+ MB


In [27]:
# Só deixa as empresas iguais

In [28]:
iguais = iguais[(iguais['Mesma_empresa'] == "Sim")]

In [29]:
iguais.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 56499 to 699481
Data columns (total 13 columns):
CNPJ_Devedor                  8 non-null object
CNPJ_Raiz                     8 non-null object
CPF_Deputado                  8 non-null object
CPF_limpo                     8 non-null int64
Nome_Deputado                 8 non-null object
Vinculo                       8 non-null object
CNPJ_limpo                    8 non-null float64
Nome_Devedor                  8 non-null object
Valores_situacao_Irregular    2 non-null float64
Valores_situacao_Regular      6 non-null float64
Total_Devido                  8 non-null float64
Total_lista_Funrural          8 non-null float64
Mesma_empresa                 8 non-null object
dtypes: float64(5), int64(1), object(7)
memory usage: 896.0+ bytes


In [30]:
iguais.reset_index()

,index,CNPJ_Devedor,CNPJ_Raiz,CPF_Deputado,CPF_limpo,Nome_Deputado,Vinculo,CNPJ_limpo,Nome_Devedor,Valores_situacao_Irregular,Valores_situacao_Regular,Total_Devido,Total_lista_Funrural,Mesma_empresa
0,56499,12.480.976/0001-98,12.480.976,010.557.374-48,1055737448,PEDRO TORRES BRANDAO VILELA,SOCIO ADMINISTRADOR,1.248098e+13,AGROPECUARIA INDIANA LTDA,NaN,53858.82,53858.82,53858.82,Sim
1,58181,07.249.030/0001-85,07.249.030,011.021.253-34,1102125334,JULIO CESAR DE CARVALHO LIMA,SOCIO ADMINISTRADOR,7.249030e+12,TROPICAL FRUTOS CANAA LTDA - ME,NaN,180940.21,180940.21,180140.12,Sim
2,81355,08.824.330/0001-03,08.824.330,012.666.376-99,1266637699,NEWTON CARDOSO JUNIOR,DIRETOR,8.824330e+12,REMPLUS EMPREENDIMENTOS E PARTICIPACOES S.A,NaN,399364.26,399364.26,369228.49,Sim
3,97980,02.486.254/0001-97,02.486.254,012.666.376-99,1266637699,NEWTON CARDOSO JUNIOR,PRESIDENTE,2.486254e+12,GOODY INDUSTRIA DE ALIMENTOS S/A,3178531.78,NaN,3178531.78,2955953.66,Sim
4,106544,17.159.559/0001-61,17.159.559,012.666.376-99,1266637699,NEWTON CARDOSO JUNIOR,PRESIDENTE,1.715956e+13,COMPANHIA SIDERURGICA PITANGUI,NaN,16346425.89,16346425.89,9086518.16,Sim
5,280957,85.466.951/0001-03,85.466.951,119.792.169-91,11979216991,DILCEU JOAO SPERAFICO,SOCIO ADMINISTRADOR,8.546695e+13,SPERAFICO INCORPORADORA LTDA - EPP,NaN,663032.79,663032.79,132844.38,Sim
6,420817,76.546.399/0001-72,76.546.399,241.063.059-68,24106305968,JACOB ALFREDO STOFFELS KAEFER,SOCIO ADMINISTRADOR,7.654640e+13,SUPER DIP DISTRIBUICAO E VAREJO LTDA,17363332.34,NaN,17363332.34,15355965.75,Sim
7,699481,12.050.989/0001-27,12.050.989,732.553.198-20,73255319820,PAULO ROBERTO GOMES MANSUR,ADMINISTRADOR,1.205099e+13,AGROTRI AGROPECUARIA TRIANGULO LTDA,NaN,39217.72,39217.72,39217.72,Sim


In [31]:
# Cria base única e salva CSV, que depois pode ser limpo

In [32]:
dividas_unidas = pd.merge(dividas, iguais, left_on='CNPJ_Devedor', right_on='CNPJ_Devedor', how = 'outer')

In [33]:
dividas_unidas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 29 columns):
CPF_Deputado_x                  96 non-null object
CPF_limpo_x                     96 non-null int64
Nome_Deputado_x                 96 non-null object
Vinculo_x                       96 non-null object
CNPJ_Devedor                    96 non-null object
CNPJ_limpo_x                    96 non-null float64
Nome_Devedor_x                  96 non-null object
Valores_situacao_Irregular_x    59 non-null float64
Valores_situacao_Regular_x      47 non-null float64
Total_Devido_x                  96 non-null float64
Estado                          96 non-null object
Cargo                           96 non-null object
Nome_candidato                  96 non-null object
CPF                             96 non-null int64
Nome_urna                       96 non-null object
Partido                         96 non-null object
Situacao                        96 non-null object
CNPJ_Raiz                   

In [34]:
dividas_unidas.to_csv('dividas_unidas.csv')